In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=25.0&md=05&md=06&md=07&md=08&md=09&md=10&ts=1&et=10&mb=0&mt=9999999&cn=20&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113"
html = requests.get(base_url)
soup = BeautifulSoup(html.content, "html.parser")

pagination = soup.find("div", class_="pagination pagination_set-nav")
last_page_link = pagination.find_all("a")[-2]
last_page_number = int(last_page_link.text)

data = []

for page_number in range(1, last_page_number + 1):
    page_url = f"{base_url}&page={page_number}"
    page_html = requests.get(page_url)
    page_soup = BeautifulSoup(page_html.content, "html.parser")

    mansions = page_soup.select("div.cassetteitem")
    for mansion in mansions:
        name = mansion.find("div", class_="cassetteitem_content-title").get_text(strip=True)
        address = mansion.find("li", class_="cassetteitem_detail-col1").get_text(strip=True)
        year = mansion.find("li", class_="cassetteitem_detail-col3").div.get_text(strip=True)
        
        rooms = mansion.select("tr.js-cassette_link")
        for room in rooms:
            floor = room.find("td", string=True).get_text(strip=True)
            rent = room.find("span", class_="cassetteitem_price--rent").get_text(strip=True)
            administration = room.find("span", class_="cassetteitem_price--administration").get_text(strip=True)
            deposit = room.find("span", class_="cassetteitem_price--deposit").get_text(strip=True)
            gratuity = room.find("span", class_="cassetteitem_price--gratuity").get_text(strip=True)
            madori = room.find("span", class_="cassetteitem_madori").get_text(strip=True)
            menseki = room.find("span", class_="cassetteitem_menseki").get_text(strip=True)

            data.append({
                '物件名': name,
                '住所': address,
                '築年数': year,
                '階': floor,
                '家賃': rent,
                '管理費': administration,
                '敷金': deposit,
                '礼金': gratuity,
                '間取り': madori,
                '専有面積': menseki,
            })

df = pd.DataFrame(data)

In [30]:
df

,物件名,住所,築年数,階,家賃,管理費,敷金,礼金,間取り,専有面積
0,kif,東京都新宿区若宮町,築15年,3階,25万円,7000円,50万円,-,3K,51.79m2
1,アヴァンティーク銀座２丁目参番館,東京都中央区銀座２,築19年,9階,20万円,10000円,20万円,20万円,2DK,42.84m2
2,銀座レジデンス伍番館,東京都中央区湊２,築20年,2階,19.2万円,-,19.2万円,19.2万円,2DK,50.38m2
3,銀座レジデンス伍番館,東京都中央区湊２,築20年,4階,19.5万円,-,19.5万円,19.5万円,2DK,50.38m2
4,銀座レジデンス伍番館,東京都中央区湊２,築20年,11階,21万円,-,21万円,21万円,2DK,52.21m2
...,...,...,...,...,...,...,...,...,...,...
1008,東京メトロ南北線 白金高輪駅 地下1地上4階建 築5年,東京都港区白金６,築5年,B1-1階,22.9万円,10000円,22.9万円,22.9万円,2LDK,55.44m2
1009,プラチナコート南麻布,東京都港区南麻布１,築19年,3階,24.1万円,10000円,-,-,2LDK,53.62m2
1010,東京メトロ南北線 麻布十番駅 8階建 築19年,東京都港区南麻布１,築19年,3階,24.1万円,10000円,-,-,2LDK,53.62m2
1011,都営大江戸線 若松河田駅 6階建 築9年,東京都新宿区河田町,築9年,5階,23万円,10000円,23万円,23万円,2LDK,53.48m2


In [ ]:
# スプレッドシート形式で出力する
df.to_excel('output3.xlsx', index=False)  # index=Falseでインデックスを出力しないようにしています

In [31]:
# '物件名', '住所', '築年数', '階', '家賃'の5つが同一である行を重複とみなして削除する
df.drop_duplicates(subset=['物件名', '住所', '築年数', '階', '家賃'], keep='first', inplace=True)

In [32]:
df

,物件名,住所,築年数,階,家賃,管理費,敷金,礼金,間取り,専有面積
0,kif,東京都新宿区若宮町,築15年,3階,25万円,7000円,50万円,-,3K,51.79m2
1,アヴァンティーク銀座２丁目参番館,東京都中央区銀座２,築19年,9階,20万円,10000円,20万円,20万円,2DK,42.84m2
2,銀座レジデンス伍番館,東京都中央区湊２,築20年,2階,19.2万円,-,19.2万円,19.2万円,2DK,50.38m2
3,銀座レジデンス伍番館,東京都中央区湊２,築20年,4階,19.5万円,-,19.5万円,19.5万円,2DK,50.38m2
4,銀座レジデンス伍番館,東京都中央区湊２,築20年,11階,21万円,-,21万円,21万円,2DK,52.21m2
...,...,...,...,...,...,...,...,...,...,...
1008,東京メトロ南北線 白金高輪駅 地下1地上4階建 築5年,東京都港区白金６,築5年,B1-1階,22.9万円,10000円,22.9万円,22.9万円,2LDK,55.44m2
1009,プラチナコート南麻布,東京都港区南麻布１,築19年,3階,24.1万円,10000円,-,-,2LDK,53.62m2
1010,東京メトロ南北線 麻布十番駅 8階建 築19年,東京都港区南麻布１,築19年,3階,24.1万円,10000円,-,-,2LDK,53.62m2
1011,都営大江戸線 若松河田駅 6階建 築9年,東京都新宿区河田町,築9年,5階,23万円,10000円,23万円,23万円,2LDK,53.48m2


In [33]:
# '築年数'列の値を変換する関数
def convert_age_of_building(age):
    if age == '新築':
        return 0
    else:
        # '築'と'年'以外の文字を取り除いて数値に変換する
        return int(age.replace('築', '').replace('年', ''))

# '築年数'列を変換する
df['築年数'] = df['築年数'].apply(convert_age_of_building)

In [34]:
df

,物件名,住所,築年数,階,家賃,管理費,敷金,礼金,間取り,専有面積
0,kif,東京都新宿区若宮町,15,3階,25万円,7000円,50万円,-,3K,51.79m2
1,アヴァンティーク銀座２丁目参番館,東京都中央区銀座２,19,9階,20万円,10000円,20万円,20万円,2DK,42.84m2
2,銀座レジデンス伍番館,東京都中央区湊２,20,2階,19.2万円,-,19.2万円,19.2万円,2DK,50.38m2
3,銀座レジデンス伍番館,東京都中央区湊２,20,4階,19.5万円,-,19.5万円,19.5万円,2DK,50.38m2
4,銀座レジデンス伍番館,東京都中央区湊２,20,11階,21万円,-,21万円,21万円,2DK,52.21m2
...,...,...,...,...,...,...,...,...,...,...
1008,東京メトロ南北線 白金高輪駅 地下1地上4階建 築5年,東京都港区白金６,5,B1-1階,22.9万円,10000円,22.9万円,22.9万円,2LDK,55.44m2
1009,プラチナコート南麻布,東京都港区南麻布１,19,3階,24.1万円,10000円,-,-,2LDK,53.62m2
1010,東京メトロ南北線 麻布十番駅 8階建 築19年,東京都港区南麻布１,19,3階,24.1万円,10000円,-,-,2LDK,53.62m2
1011,都営大江戸線 若松河田駅 6階建 築9年,東京都新宿区河田町,9,5階,23万円,10000円,23万円,23万円,2LDK,53.48m2


In [37]:
def convert_floor(floor):
    return str(floor.replace('階', ''))
df['階'] = df['階'].apply(convert_floor)

In [38]:
df

,物件名,住所,築年数,階,家賃,管理費,敷金,礼金,間取り,専有面積
0,kif,東京都新宿区若宮町,15,3,25万円,7000円,50万円,-,3K,51.79m2
1,アヴァンティーク銀座２丁目参番館,東京都中央区銀座２,19,9,20万円,10000円,20万円,20万円,2DK,42.84m2
2,銀座レジデンス伍番館,東京都中央区湊２,20,2,19.2万円,-,19.2万円,19.2万円,2DK,50.38m2
3,銀座レジデンス伍番館,東京都中央区湊２,20,4,19.5万円,-,19.5万円,19.5万円,2DK,50.38m2
4,銀座レジデンス伍番館,東京都中央区湊２,20,11,21万円,-,21万円,21万円,2DK,52.21m2
...,...,...,...,...,...,...,...,...,...,...
1008,東京メトロ南北線 白金高輪駅 地下1地上4階建 築5年,東京都港区白金６,5,B1-1,22.9万円,10000円,22.9万円,22.9万円,2LDK,55.44m2
1009,プラチナコート南麻布,東京都港区南麻布１,19,3,24.1万円,10000円,-,-,2LDK,53.62m2
1010,東京メトロ南北線 麻布十番駅 8階建 築19年,東京都港区南麻布１,19,3,24.1万円,10000円,-,-,2LDK,53.62m2
1011,都営大江戸線 若松河田駅 6階建 築9年,東京都新宿区河田町,9,5,23万円,10000円,23万円,23万円,2LDK,53.48m2


In [42]:

# '家賃'列を数値に変換する関数
def convert_rent(rent):
        # '万円'を取り除いて数値に変換する
    return float(rent.replace('万円', ''))
df['家賃'] = df['家賃'].apply(convert_rent)

In [43]:
df

,物件名,住所,築年数,階,家賃,管理費,敷金,礼金,間取り,専有面積
0,kif,東京都新宿区若宮町,15,3,25.0,7000円,50万円,-,3K,51.79m2
1,アヴァンティーク銀座２丁目参番館,東京都中央区銀座２,19,9,20.0,10000円,20万円,20万円,2DK,42.84m2
2,銀座レジデンス伍番館,東京都中央区湊２,20,2,19.2,-,19.2万円,19.2万円,2DK,50.38m2
3,銀座レジデンス伍番館,東京都中央区湊２,20,4,19.5,-,19.5万円,19.5万円,2DK,50.38m2
4,銀座レジデンス伍番館,東京都中央区湊２,20,11,21.0,-,21万円,21万円,2DK,52.21m2
...,...,...,...,...,...,...,...,...,...,...
1008,東京メトロ南北線 白金高輪駅 地下1地上4階建 築5年,東京都港区白金６,5,B1-1,22.9,10000円,22.9万円,22.9万円,2LDK,55.44m2
1009,プラチナコート南麻布,東京都港区南麻布１,19,3,24.1,10000円,-,-,2LDK,53.62m2
1010,東京メトロ南北線 麻布十番駅 8階建 築19年,東京都港区南麻布１,19,3,24.1,10000円,-,-,2LDK,53.62m2
1011,都営大江戸線 若松河田駅 6階建 築9年,東京都新宿区河田町,9,5,23.0,10000円,23万円,23万円,2LDK,53.48m2


In [46]:

def convert_administration(administration):
    if administration == '-':
        return 0
    else:
        administration = float(administration.replace('円', '')) / 10000
        return administration

df['管理費'] = df['管理費'].apply(convert_administration)

In [47]:
df

,物件名,住所,築年数,階,家賃,管理費,敷金,礼金,間取り,専有面積
0,kif,東京都新宿区若宮町,15,3,25.0,0.7,50万円,-,3K,51.79m2
1,アヴァンティーク銀座２丁目参番館,東京都中央区銀座２,19,9,20.0,1.0,20万円,20万円,2DK,42.84m2
2,銀座レジデンス伍番館,東京都中央区湊２,20,2,19.2,0.0,19.2万円,19.2万円,2DK,50.38m2
3,銀座レジデンス伍番館,東京都中央区湊２,20,4,19.5,0.0,19.5万円,19.5万円,2DK,50.38m2
4,銀座レジデンス伍番館,東京都中央区湊２,20,11,21.0,0.0,21万円,21万円,2DK,52.21m2
...,...,...,...,...,...,...,...,...,...,...
1008,東京メトロ南北線 白金高輪駅 地下1地上4階建 築5年,東京都港区白金６,5,B1-1,22.9,1.0,22.9万円,22.9万円,2LDK,55.44m2
1009,プラチナコート南麻布,東京都港区南麻布１,19,3,24.1,1.0,-,-,2LDK,53.62m2
1010,東京メトロ南北線 麻布十番駅 8階建 築19年,東京都港区南麻布１,19,3,24.1,1.0,-,-,2LDK,53.62m2
1011,都営大江戸線 若松河田駅 6階建 築9年,東京都新宿区河田町,9,5,23.0,1.0,23万円,23万円,2LDK,53.48m2


In [49]:

def convert_deposit(deposit):
    if deposit == '-':
        return 0
    return float(deposit.replace('万円', ''))
df['敷金'] = df['敷金'].apply(convert_deposit)

In [52]:
df.dtypes

物件名      object
住所       object
築年数       int64
階        object
家賃      float64
管理費     float64
敷金      float64
礼金      float64
間取り      object
専有面積     object
dtype: object

In [53]:
df

,物件名,住所,築年数,階,家賃,管理費,敷金,礼金,間取り,専有面積
0,kif,東京都新宿区若宮町,15,3,25.0,0.7,50.0,0.0,3K,51.79m2
1,アヴァンティーク銀座２丁目参番館,東京都中央区銀座２,19,9,20.0,1.0,20.0,20.0,2DK,42.84m2
2,銀座レジデンス伍番館,東京都中央区湊２,20,2,19.2,0.0,19.2,19.2,2DK,50.38m2
3,銀座レジデンス伍番館,東京都中央区湊２,20,4,19.5,0.0,19.5,19.5,2DK,50.38m2
4,銀座レジデンス伍番館,東京都中央区湊２,20,11,21.0,0.0,21.0,21.0,2DK,52.21m2
...,...,...,...,...,...,...,...,...,...,...
1008,東京メトロ南北線 白金高輪駅 地下1地上4階建 築5年,東京都港区白金６,5,B1-1,22.9,1.0,22.9,22.9,2LDK,55.44m2
1009,プラチナコート南麻布,東京都港区南麻布１,19,3,24.1,1.0,0.0,0.0,2LDK,53.62m2
1010,東京メトロ南北線 麻布十番駅 8階建 築19年,東京都港区南麻布１,19,3,24.1,1.0,0.0,0.0,2LDK,53.62m2
1011,都営大江戸線 若松河田駅 6階建 築9年,東京都新宿区河田町,9,5,23.0,1.0,23.0,23.0,2LDK,53.48m2


In [57]:

def convert_gratuity(gratuity):
    if gratuity == '-':
        return 0
    return float(gratuity.replace('万円', ''))


In [58]:
df

,物件名,住所,築年数,階,家賃,管理費,敷金,礼金,間取り,専有面積
0,kif,東京都新宿区若宮町,15,3,25.0,0.7,50.0,0.0,3K,51.79m2
1,アヴァンティーク銀座２丁目参番館,東京都中央区銀座２,19,9,20.0,1.0,20.0,20.0,2DK,42.84m2
2,銀座レジデンス伍番館,東京都中央区湊２,20,2,19.2,0.0,19.2,19.2,2DK,50.38m2
3,銀座レジデンス伍番館,東京都中央区湊２,20,4,19.5,0.0,19.5,19.5,2DK,50.38m2
4,銀座レジデンス伍番館,東京都中央区湊２,20,11,21.0,0.0,21.0,21.0,2DK,52.21m2
...,...,...,...,...,...,...,...,...,...,...
1008,東京メトロ南北線 白金高輪駅 地下1地上4階建 築5年,東京都港区白金６,5,B1-1,22.9,1.0,22.9,22.9,2LDK,55.44m2
1009,プラチナコート南麻布,東京都港区南麻布１,19,3,24.1,1.0,0.0,0.0,2LDK,53.62m2
1010,東京メトロ南北線 麻布十番駅 8階建 築19年,東京都港区南麻布１,19,3,24.1,1.0,0.0,0.0,2LDK,53.62m2
1011,都営大江戸線 若松河田駅 6階建 築9年,東京都新宿区河田町,9,5,23.0,1.0,23.0,23.0,2LDK,53.48m2


In [66]:

def convert_menseki(menseki):
    menseki = float(menseki.replace('m2', ''))
    return menseki
df['専有面積'] = df['専有面積'].apply(convert_menseki)

In [67]:
df

,物件名,住所,築年数,階,家賃,管理費,敷金,礼金,間取り,専有面積
0,kif,東京都新宿区若宮町,15,3,25.0,0.7,50.0,0.0,3K,51.79
1,アヴァンティーク銀座２丁目参番館,東京都中央区銀座２,19,9,20.0,1.0,20.0,20.0,2DK,42.84
2,銀座レジデンス伍番館,東京都中央区湊２,20,2,19.2,0.0,19.2,19.2,2DK,50.38
3,銀座レジデンス伍番館,東京都中央区湊２,20,4,19.5,0.0,19.5,19.5,2DK,50.38
4,銀座レジデンス伍番館,東京都中央区湊２,20,11,21.0,0.0,21.0,21.0,2DK,52.21
...,...,...,...,...,...,...,...,...,...,...
1008,東京メトロ南北線 白金高輪駅 地下1地上4階建 築5年,東京都港区白金６,5,B1-1,22.9,1.0,22.9,22.9,2LDK,55.44
1009,プラチナコート南麻布,東京都港区南麻布１,19,3,24.1,1.0,0.0,0.0,2LDK,53.62
1010,東京メトロ南北線 麻布十番駅 8階建 築19年,東京都港区南麻布１,19,3,24.1,1.0,0.0,0.0,2LDK,53.62
1011,都営大江戸線 若松河田駅 6階建 築9年,東京都新宿区河田町,9,5,23.0,1.0,23.0,23.0,2LDK,53.48


In [68]:

# '家賃'列のタイトルを変更する
df.rename(columns={'家賃': '家賃[万円]'}, inplace=True)
df.rename(columns={'管理費': '管理費[万円]'}, inplace=True)
df.rename(columns={'敷金': '敷金[万円]'}, inplace=True)
df.rename(columns={'礼金': '礼金[万円]'}, inplace=True)
df.rename(columns={'専有面積': '専有面積[m2]'}, inplace=True)

In [69]:
df

,物件名,住所,築年数,階,家賃[万円],管理費[万円],敷金[万円],礼金[万円],間取り,専有面積[m2]
0,kif,東京都新宿区若宮町,15,3,25.0,0.7,50.0,0.0,3K,51.79
1,アヴァンティーク銀座２丁目参番館,東京都中央区銀座２,19,9,20.0,1.0,20.0,20.0,2DK,42.84
2,銀座レジデンス伍番館,東京都中央区湊２,20,2,19.2,0.0,19.2,19.2,2DK,50.38
3,銀座レジデンス伍番館,東京都中央区湊２,20,4,19.5,0.0,19.5,19.5,2DK,50.38
4,銀座レジデンス伍番館,東京都中央区湊２,20,11,21.0,0.0,21.0,21.0,2DK,52.21
...,...,...,...,...,...,...,...,...,...,...
1008,東京メトロ南北線 白金高輪駅 地下1地上4階建 築5年,東京都港区白金６,5,B1-1,22.9,1.0,22.9,22.9,2LDK,55.44
1009,プラチナコート南麻布,東京都港区南麻布１,19,3,24.1,1.0,0.0,0.0,2LDK,53.62
1010,東京メトロ南北線 麻布十番駅 8階建 築19年,東京都港区南麻布１,19,3,24.1,1.0,0.0,0.0,2LDK,53.62
1011,都営大江戸線 若松河田駅 6階建 築9年,東京都新宿区河田町,9,5,23.0,1.0,23.0,23.0,2LDK,53.48


In [70]:
df.drop_duplicates(subset=['住所', '築年数', '階', '家賃[万円]'], keep='first', inplace=True)

In [71]:
df

,物件名,住所,築年数,階,家賃[万円],管理費[万円],敷金[万円],礼金[万円],間取り,専有面積[m2]
0,kif,東京都新宿区若宮町,15,3,25.0,0.7,50.0,0.0,3K,51.79
1,アヴァンティーク銀座２丁目参番館,東京都中央区銀座２,19,9,20.0,1.0,20.0,20.0,2DK,42.84
2,銀座レジデンス伍番館,東京都中央区湊２,20,2,19.2,0.0,19.2,19.2,2DK,50.38
3,銀座レジデンス伍番館,東京都中央区湊２,20,4,19.5,0.0,19.5,19.5,2DK,50.38
4,銀座レジデンス伍番館,東京都中央区湊２,20,11,21.0,0.0,21.0,21.0,2DK,52.21
...,...,...,...,...,...,...,...,...,...,...
1007,ラブランシュール,東京都港区白金６,5,1,22.9,1.0,22.9,22.9,2LDK,55.44
1008,東京メトロ南北線 白金高輪駅 地下1地上4階建 築5年,東京都港区白金６,5,B1-1,22.9,1.0,22.9,22.9,2LDK,55.44
1009,プラチナコート南麻布,東京都港区南麻布１,19,3,24.1,1.0,0.0,0.0,2LDK,53.62
1011,都営大江戸線 若松河田駅 6階建 築9年,東京都新宿区河田町,9,5,23.0,1.0,23.0,23.0,2LDK,53.48


In [73]:
# スプレッドシート形式で出力する
df.to_excel('output4.xlsx', index=False)  # index=Falseでインデックスを出力しないようにしています

In [81]:
pip install gspread

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [83]:
import json
import pandas as pd

In [84]:
pip install oauth2client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [95]:
from oauth2client.service_account import ServiceAccountCredentials

In [97]:
SP_CREDENTIAL_FILE = 'gspread-test-step3-267b4903a38a.json'
SP_COPE = [
    'https://www.googleapis.com/auth/drive',
    'https://spreadsheets.google.com/feeds'
]
SP_SHEET_KEY = '18ajvVjBf3SW3DvPZWACtXRc-Adi87ObV6TXSULhK4f8'
SP_SHEET = 'demo'

In [99]:
import gspread

In [100]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(SP_CREDENTIAL_FILE, SP_COPE)
gc = gspread.authorize(credentials)

In [101]:
sh = gc.open_by_key(SP_SHEET_KEY)

In [102]:
worksheet = sh.worksheet(SP_SHEET)

In [103]:
data = worksheet.get_all_values()
data

[['物件名',
  '住所',
  '築年数',
  '階',
  '家賃[万円]',
  '管理費[万円]',
  '敷金[万円]',
  '礼金[万円]',
  '間取り',
  '専有面積[m2]'],
 ['kif', '東京都新宿区若宮町', '15', '3', '25', '0.7', '50', '0', '3K', '51.79'],
 ['アヴァンティーク銀座２丁目参番館',
  '東京都中央区銀座２',
  '19',
  '9',
  '20',
  '1',
  '20',
  '20',
  '2DK',
  '42.84'],
 ['銀座レジデンス伍番館',
  '東京都中央区湊２',
  '20',
  '2',
  '19.2',
  '0',
  '19.2',
  '19.2',
  '2DK',
  '50.38'],
 ['銀座レジデンス伍番館',
  '東京都中央区湊２',
  '20',
  '4',
  '19.5',
  '0',
  '19.5',
  '19.5',
  '2DK',
  '50.38'],
 ['銀座レジデンス伍番館', '東京都中央区湊２', '20', '11', '21', '0', '21', '21', '2DK', '52.21'],
 ['kinomori代々木上原', '東京都渋谷区上原２', '1', '4', '12.7', '1', '0', '0', '2K', '23.7'],
 ['kinomori代々木上原',
  '東京都渋谷区上原２',
  '1',
  '2',
  '13.1',
  '1',
  '0',
  '0',
  '2K',
  '24.08'],
 ['kinomori代々木上原',
  '東京都渋谷区上原２',
  '1',
  '3',
  '13.2',
  '1',
  '0',
  '0',
  '2K',
  '24.08'],
 ['kinomori代々木上原', '東京都渋谷区上原２', '1', '1', '14.3', '1', '0', '0', '2K', '27.5'],
 ['kinomori代々木上原', '東京都渋谷区上原２', '1', '2', '14.4', '1', '0', '0', '2K', '2

In [104]:
df = pd.DataFrame(data[1:], columns=data[0])
df

,物件名,住所,築年数,階,家賃[万円],管理費[万円],敷金[万円],礼金[万円],間取り,専有面積[m2]
0,kif,東京都新宿区若宮町,15,3,25,0.7,50,0,3K,51.79
1,アヴァンティーク銀座２丁目参番館,東京都中央区銀座２,19,9,20,1,20,20,2DK,42.84
2,銀座レジデンス伍番館,東京都中央区湊２,20,2,19.2,0,19.2,19.2,2DK,50.38
3,銀座レジデンス伍番館,東京都中央区湊２,20,4,19.5,0,19.5,19.5,2DK,50.38
4,銀座レジデンス伍番館,東京都中央区湊２,20,11,21,0,21,21,2DK,52.21
...,...,...,...,...,...,...,...,...,...,...
599,ラブランシュール,東京都港区白金６,5,1,22.9,1,22.9,22.9,2LDK,55.44
600,東京メトロ南北線 白金高輪駅 地下1地上4階建 築5年,東京都港区白金６,5,B1-1,22.9,1,22.9,22.9,2LDK,55.44
601,プラチナコート南麻布,東京都港区南麻布１,19,3,24.1,1,0,0,2LDK,53.62
602,都営大江戸線 若松河田駅 6階建 築9年,東京都新宿区河田町,9,5,23,1,23,23,2LDK,53.48
